In [4]:
import pandas as pd

bit_url = 'https://raw.githubusercontent.com/Itisbeen/Data_Analysis_project/refs/heads/main/csv_file/BTC_USD%20%EA%B3%BC%EA%B1%B0%20%EB%8D%B0%EC%9D%B4%ED%84%B0.csv'
nasdaq_url = 'https://raw.githubusercontent.com/Itisbeen/Data_Analysis_project/refs/heads/main/csv_file/%EB%82%98%EC%8A%A4%EB%8B%A5%20100%20%EA%B3%BC%EA%B1%B0%20%EB%8D%B0%EC%9D%B4%ED%84%B0.csv'
oil_url = 'https://raw.githubusercontent.com/Itisbeen/Data_Analysis_project/refs/heads/main/csv_file/WTI%EC%9C%A0%20%EC%84%A0%EB%AC%BC%20%EA%B3%BC%EA%B1%B0%20%EB%8D%B0%EC%9D%B4%ED%84%B0.csv'
gold_url = 'https://raw.githubusercontent.com/Itisbeen/Data_Analysis_project/refs/heads/main/csv_file/%EA%B8%88%20%EC%84%A0%EB%AC%BC%20%EA%B3%BC%EA%B1%B0%20%EB%8D%B0%EC%9D%B4%ED%84%B0.csv'


df_bitcoin = pd.read_csv(bit_url)
df_nasdaq = pd.read_csv(nasdaq_url)
df_gold = pd.read_csv(gold_url)
df_oil = pd.read_csv(oil_url)

df_bitcoin.head()

,날짜,종가,시가,고가,저가,거래량,변동 %
0,2025- 04- 27,"94,232.0","94,693.0","95,369.0","94,144.0",0.31K,-0.49%
1,2025- 04- 26,"94,693.0","94,707.0","95,228.0","94,083.0",0.31K,-0.01%
2,2025- 04- 25,"94,702.0","93,940.0","95,800.0","92,921.0",0.71K,0.81%
3,2025- 04- 24,"93,940.0","93,639.0","94,000.0","91,760.0",0.34K,0.32%
4,2025- 04- 23,"93,639.0","93,303.0","94,415.0","91,919.0",1.30K,0.36%
